RAW (MOABB) to CSV

This code convert the data sets from RAW format to CSV format using MOABB.

It has been specifically conceived for BCI data.

This script is for EPFLP300-6 (Radio run 6)



In [ ]:
import numpy as np
import pandas as pd
import moabb.datasets

In [ ]:
#Load Database
m_dataset = moabb.datasets.Zhou2016()
m_data = m_dataset.get_data()

In [ ]:
#See all canal names (EEG, misc, stim...)
raws = m_data[1]['0']['1']
raws
print("Canal list :", raws.ch_names)

In [ ]:
#Know what index is stim channel (we will need it later for the CSV to NY conversion)
stim_channel_name = 'STIM'
stim_idx = raws.ch_names.index(stim_channel_name)
print(f"Canal index {stim_channel_name} is : {stim_idx}")


In [ ]:
#count stim data unique values (1 non target, 2 = target with a ratio needed of 5 to 1)
stim_data = raws.get_data(picks=stim_idx)
print(stim_data.shape)
unique_vals, counts = np.unique(stim_data, return_counts=True)

for val, count in zip(unique_vals, counts):
    print(f"Value : {val}, Occurences count : {count}")

In [ ]:
# Par exemple, pour le sujet 1 et une session donnée (ici '0' ou '1test' selon la nomenclature)
subject = 1
session = '0'  # ou '0' selon ton cas

# Récupérer la liste des runs dans cette session
run_keys = sorted(m_data[subject][session].keys())
print("Runs trouvés dans la session :", run_keys)

# Initialiser une liste pour stocker les données de chaque run
all_runs_data = []

# Parcourir chaque run et extraire ses données
for run in run_keys:
    raw_run = m_data[subject][session][run]
    raw_run.drop_channels(['VEOU', 'VEOL'])
    run_data = raw_run.get_data()  # forme: (n_channels, n_times_run)
    all_runs_data.append(run_data)

# Concaténer les données le long de l'axe temporel (axis=1)
concatenated_data = np.concatenate(all_runs_data, axis=1)  # forme: (n_channels, total_timesamples)

# Transposer pour avoir un tableau de forme (total_timesamples, n_channels)
dataT = concatenated_data.T

print("Forme de dataT :", dataT.shape)

In [ ]:
# Extraire la dernière colonne (canal stim)
stim_col = dataT[:, -1]

# Compter les valeurs uniques
unique_vals, counts = np.unique(stim_col, return_counts=True)

for val, count in zip(unique_vals, counts):
    print(f"Value : {val}, Occurrence count : {count}")


In [ ]:
# creating timestamps and header
n_times, n_channels = dataT.shape
timestamps = np.arange(n_times, dtype=int)
data_with_timestamp = np.column_stack((timestamps, dataT))
header = [""] + [str(i) for i in range(n_channels)]

# Removing decimals from timestamps
df = pd.DataFrame(data_with_timestamp, columns=header)
df[""] = df[""].astype(int)

In [ ]:
# Test to check csv file
df.to_csv("data.csv", index=False)

In [ ]:
subject_list = list(m_data.keys())
# Définir les sessions à traiter (ici '0train' et '0test')

for subject in subject_list:
    session_keys = sorted(m_data[subject].keys())
    for session in session_keys:
        # Récupérer la liste des runs dans cette session
        run_keys = sorted(m_data[subject][session].keys())
        
        # Initialiser une liste pour stocker les données de chaque run
        all_runs_data = []
        for run in run_keys:
            raw_run = m_data[subject][session][run]
            # Retirer les canaux EOG
            raw_run.drop_channels(['VEOU', 'VEOL'])
            run_data = raw_run.get_data()  # forme: (n_channels, n_times_run)
            all_runs_data.append(run_data)
        
        # Concaténer les données le long de l'axe temporel (axis=1)
        concatenated_data = np.concatenate(all_runs_data, axis=1)  # (n_channels, total_timesamples)
        
        # Transposer pour obtenir dataT de forme (total_timesamples, n_channels)
        dataT = concatenated_data.T
        n_times, n_channels = dataT.shape
        
        # Création de la colonne de timestamps
        timestamps = np.arange(n_times, dtype=int)
        datacsv = np.column_stack((timestamps, dataT))
        header = [""] + [str(i) for i in range(n_channels)]
        df = pd.DataFrame(datacsv, columns=header)
        df[""] = df[""].astype(int)
        
        # Nommer le fichier
        subject_str = f"{int(subject):02d}"
        # Calculer le numéro de session à partir de la chaîne '0train' ou '1test'
        session_str = f"{int(session[0]) + 1:02d}"
        filename = f"subject_{subject_str}_session_{session_str}.csv"
        df.to_csv(filename, index=False)
        print(f"Saved file : {filename}")